In [2]:
import sqlite3
import pandas as pd
import sys

sys.path.append('../')

from dotenv import load_dotenv

load_dotenv()
import os
DB_PATH = os.getenv("DB_PATH")


conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [3]:
# Load individuals Main Informations
individuals = pd.read_sql_query("SELECT * FROM individuals_kept", conn)

In [18]:
# Load identifiers meta-data
df_identifiers = pd.read_sql_query("SELECT * FROM identifiers", conn)
df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''
df_identifiers['identifier_name'] = df_identifiers.apply(lambda x : x['identifier_name'] + ' ('  + x['country_name'] + ')' if x['country_name']!='' else x['identifier_name'], axis=1)
df_identifiers

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_34462/250969630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''


,identifiers_wikidata_id,identifier_name,country_wikidata_id,country_name,count_records,identifier_url,identifier_name_country
0,P212,ISBN-13,None,,NaN,None,ISBN-13
1,P213,ISNI,None,,11020000.0,None,ISNI
2,P214,VIAF ID,None,,33656281.0,https://viaf.org,VIAF ID
3,P218,ISO 639-1 code,None,,184.0,None,ISO 639-1 code
4,P219,ISO 639-2 code,None,,486.0,None,ISO 639-2 code
...,...,...,...,...,...,...,...
8017,P11281,Biographical Dictionary of the Australian Sena...,Q408,Australia,600.0,https://biography.senate.gov.au/,Biographical Dictionary of the Australian Sena...
8018,P11282,Tanzania Parliament member ID (Tanzania),Q924,Tanzania,618.0,https://www.parliament.go.tz/mps-list,Tanzania Parliament member ID (Tanzania)
8019,P11283,JCDb ID (Japan),Q17,Japan,49095.0,https://www.japanese-cinema-db.jp/,JCDb ID (Japan)
8020,P11284,Dictionnaire de l'Académie française ID (1st e...,Q142,France,NaN,None,Dictionnaire de l'Académie française ID (1st e...


In [ ]:

df_identifiers = df_identifiers.drop(['country_wikidata_id', 'count_records', 'identifier_url'], axis=1).drop_duplicates()
df_identifiers = df_identifiers.drop(['country_name', 'identifier_name_country'], axis=1).drop_duplicates()

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_34462/4189010349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''


,identifiers_wikidata_id,identifier_name
0,P212,ISBN-13
1,P213,ISNI
2,P214,VIAF ID
3,P218,ISO 639-1 code
4,P219,ISO 639-2 code
...,...,...
8017,P11281,Biographical Dictionary of the Australian Sena...
8018,P11282,Tanzania Parliament member ID (Tanzania)
8019,P11283,JCDb ID (Japan)
8020,P11284,Dictionnaire de l'Académie française ID (1st e...


In [15]:
# Load infirmation about individuals and identifiers
df_ind_identifiers = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
df_ind_identifiers = df_ind_identifiers.drop('identifier_name', axis=1)
df_ind_identifiers = pd.merge(df_ind_identifiers, df_identifiers, on = 'identifiers_wikidata_id', how = 'left')

# Freebase is now mixed with Google ID
df_ind_identifiers['identifier_name'][df_ind_identifiers['identifier_name']=='Freebase ID'] = 'Google Knowledge Graph ID'
df_ind_identifiers['identifiers_wikidata_id'][df_ind_identifiers['identifiers_wikidata_id']=='P646'] = 'P2671'

# Mix the two GND
df_ind_identifiers['identifier_name'][df_ind_identifiers['identifier_name']=='Deutsche Biographie (GND) ID (Germany)'] = 'GND ID (Germany)'
df_ind_identifiers['identifiers_wikidata_id'][df_ind_identifiers['identifiers_wikidata_id']=='P7902'] = 'P227'

# Clean wrong objects
df_ind_identifiers = df_ind_identifiers[~df_ind_identifiers['individual_name'].str.contains('Painter')]

# Remove superceded
df_ind_identifiers = df_ind_identifiers[~df_ind_identifiers['identifier_name'].str.contains('superceded')]
df_ind_identifiers = df_ind_identifiers[~df_ind_identifiers['identifier_name'].str.contains('obsolete')]

# Drop duplicated after mergeing GND and Freebase
df_ind_identifiers = df_ind_identifiers.drop_duplicates()

,individual_wikidata_id,individual_name,identifiers_wikidata_id,identifier_name
0,Q46995803,Johann Michael Vogt,P214,VIAF ID
1,Q46995803,Johann Michael Vogt,P227,GND ID (Germany)
2,Q46995803,Johann Michael Vogt,P2671,Google Knowledge Graph ID
3,Q46995803,Johann Michael Vogt,P3413,Leopoldina member ID (superseded) (Germany)
4,Q4684802,Adriaen Matham,P213,ISNI
...,...,...,...,...
3003267,Q60214671,Joseph Schneller,P9223,Provenio UUID (Czech Republic)
3003268,Q97571271,Antonio González de Cedillo,P245,Union List of Artist Names ID (United States o...
3003269,Q97571271,Antonio González de Cedillo,P2843,Benezit ID
3003270,Q97571271,Antonio González de Cedillo,P7848,Frick Art Reference Library Artist File ID


In [6]:
len(set(df_ind_identifiers.identifiers_wikidata_id))

1672

In [7]:
df_final = pd.merge(df_ind_identifiers, individuals, on = 'individual_wikidata_id')
df_stats = df_final[['individual_wikidata_id', 'identifier_name']].drop_duplicates()



df_filtered = pd.read_csv('..//3-unseen_species_index/data/CPs_in_final_dataset.csv')
df_stats = df_stats[df_stats['individual_wikidata_id'].isin(df_filtered['individual_wikidata_id'])]
df_stats


,individual_wikidata_id,identifier_name
0,Q46995803,VIAF ID
1,Q46995803,GND ID (Germany)
2,Q46995803,Google Knowledge Graph ID
3,Q46995803,Leopoldina member ID (superseded) (Germany)
4,Q4684802,ISNI
...,...,...
2525780,Q105808359,AKL Online artist ID
2525791,Q97571271,Union List of Artist Names ID (United States o...
2525792,Q97571271,Benezit ID
2525793,Q97571271,Frick Art Reference Library Artist File ID


In [8]:
len(set(df_stats['identifier_name']))

1521

In [9]:
len(list(set(df_stats.identifier_name)))

1521

In [10]:
# Group by 'identifier_name' and calculate the count and percentage
grouped = df_stats.groupby('identifier_name').size().reset_index(name='count')
grouped = grouped.sort_values('count', ascending=False).reset_index(drop=True)

grouped['percent'] = round(grouped['count']/len(df_filtered)*100, 1)


# grouped['percent'] = round(grouped['count']/len(individuals)*100, 1)
grouped.columns = ['Catalog', 'N Individuals', '%']

grouped['Catalog'] = grouped['Catalog'].str.replace('United States of America', 'US')
grouped.head(20)

,Catalog,N Individuals,%
0,VIAF ID,101591,82.8
1,Google Knowledge Graph ID,91061,74.3
2,GND ID (Germany),75873,61.9
3,ISNI,73498,59.9
4,CERL Thesaurus ID,61182,49.9
5,Library of Congress authority ID (US),54282,44.3
6,IdRef ID (France),41309,33.7
7,Bibliothèque nationale de France ID (France),40380,32.9
8,Nationale Thesaurus voor Auteurs ID (Netherlands),38581,31.5
9,FAST ID,31412,25.6


,individual_wikidata_id,identifier_name
0,Q46995803,VIAF ID
1,Q46995803,GND ID (Germany)
2,Q46995803,Google Knowledge Graph ID
3,Q46995803,Leopoldina member ID (superseded) (Germany)
4,Q4684802,ISNI
...,...,...
2525780,Q105808359,AKL Online artist ID
2525791,Q97571271,Union List of Artist Names ID (United States o...
2525792,Q97571271,Benezit ID
2525793,Q97571271,Frick Art Reference Library Artist File ID


In [11]:
# Group by 'identifier_name' and calculate the count and percentage
grouped = df_stats.groupby('identifier_name').size().reset_index(name='count')
grouped = grouped.sort_values('count', ascending=False).reset_index(drop=True)

grouped['percent'] = round(grouped['count']/len(df_filtered)*100, 1)

# grouped['percent'] = round(grouped['count']/len(individuals)*100, 1)
grouped.columns = ['Catalog', 'N Individuals', '%']

grouped['Catalog'] = grouped['Catalog'].str.replace('United States of America', 'US')

# Add total row
total_row = pd.DataFrame({
    'Catalog': ['Total'],
    'N Individuals': [len(df_filtered)],
    '%': [100.0]
})

grouped = pd.concat([grouped.head(30), total_row], ignore_index=True)
grouped

,Catalog,N Individuals,%
0,VIAF ID,101591,82.8
1,Google Knowledge Graph ID,91061,74.3
2,GND ID (Germany),75873,61.9
3,ISNI,73498,59.9
4,CERL Thesaurus ID,61182,49.9
5,Library of Congress authority ID (US),54282,44.3
6,IdRef ID (France),41309,33.7
7,Bibliothèque nationale de France ID (France),40380,32.9
8,Nationale Thesaurus voor Auteurs ID (Netherlands),38581,31.5
9,FAST ID,31412,25.6


In [12]:
# ind_unseen = pd.read_csv('unseen_species_model/unseen_species_model_bayesian.csv')
# df_stats_filtered = df_stats[df_stats['individual_wikidata_id'].isin(list(set(ind_unseen['individual_wikidata_id'])))]
# print(len(set(df_stats_filtered['identifier_name'])))


# df_identifiers = pd.read_sql_query("SELECT * FROM identifiers", conn)
# df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''
# df_identifiers['identifier_name'] = df_identifiers.apply(lambda x : x['identifier_name'] + ' ('  + x['country_name'] + ')' if x['country_name']!='' else x['identifier_name'], axis=1)


# # Load infirmation about individuals and identifiers
# df_ind_identifiers = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
# df_ind_identifiers = df_ind_identifiers.drop('identifier_name', axis=1)
# df_ind_identifiers = pd.merge(df_ind_identifiers, df_identifiers, on = 'identifiers_wikidata_id', how = 'left')
# df_ind_identifiers

In [13]:
# df_stats_filtered = pd.merge(df_stats_filtered, df_identifiers, on = 'identifier_name')
# df_stats_filtered